In [1]:
import pandas as pd
import os
from glob import glob
import re
import requests
import shutil
import dotenv

In [ ]:
dotenv.load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")

In [2]:
import openai
openai.api_key=OPENAI_API_KEY
openai.organization=OPENAI_ORGANIZATION
model_name="gpt-3.5-turbo"

In [3]:
base_prompt = """In the sentence below, give me the list of:
- organization/University/College/Institute named entity
- location named entity
- Country named entity
- miscellaneous named entity.
Format the output in json with the following keys:
- ORGANIZATION for organization named entity
- LOCATION for location named entity
- COUNTRY for person named entity
- MISCELLANEOUS for miscellaneous named entity.
Sentence below:
"""

def ask_openai(prompt: str) -> str:
    try:
        message = {
            'role': 'user',
            'content': prompt
        }
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[message]
        )
        return response.choices[0].message['content']
    except Exception as e:
        print("Error:", e)
        return ""

# print(ask_openai(base_prompt +  "Elon Musk is the CEO of Tesla and SpaceX. He was born in South Africa and now lives in the"
#     " USA. He is one of the founders of OpenAI."))

In [4]:
!pip install openai
# 

In [5]:
import pandas as pd
from bs4 import BeautifulSoup # BeautifulSoup is in bs4 package 
import requests
# import browsercookie
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import csv
import time
import re
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox") # linux only
chrome_options.add_argument("--headless")
chrome_options.add_argument('disable-blink-features=AutomationControlled')
chrome_options.add_argument('user-agent=fake-useragent')
chrome_options.add_argument('--user-data-dir=~/.config/google-chrome')
chrome_options.headless = True # also works
driver = webdriver.Chrome(options=chrome_options)


In [6]:
file_ = pd.read_csv('eccv_scopus.csv',low_memory=False)

In [7]:
file_sample =file_.head()

In [8]:
file_sample

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,Ignatov A.; Timofte R.; Zhang J.; Zhang F.; Yu...,"Ignatov, Andrey (57194704046); Timofte, Radu (...",57194704046; 35812778400; 57658328700; 5720777...,Realistic Bokeh Effect Rendering on Mobile GPU...,2023,Lecture Notes in Computer Science (including s...,13803 LNCS,NaN,NaN,153,...,978-303125065-1,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85151126246
1,Jiang Y.; Wang Y.; Zhang R.; Xu Q.; Zhang Y.; ...,"Jiang, Yuxuan (58404466900); Wang, Yanfeng (57...",58404466900; 57188755369; 57216507923; 5721650...,Domain-Conditioned Normalization for Test-Time...,2023,Lecture Notes in Computer Science (including s...,13808 LNCS,NaN,NaN,291,...,978-303125084-2,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85151325604
2,Vázquez Enríquez M.; Castro J.L.A.; Fernandez ...,"Vázquez Enríquez, Manuel (57266982000); Castro...",57266982000; 57198788259; 57192183772; 5720118...,"ECCV 2022 Sign Spotting Challenge: Dataset, De...",2023,Lecture Notes in Computer Science (including s...,13808 LNCS,NaN,NaN,225,...,978-303125084-2,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85151337313
3,Li C.; Wu B.; Pumarola A.; Zhang P.; Lin Y.; V...,"Li, Chaojian (57218491194); Wu, Bichen (571949...",57218491194; 57194943616; 56507080500; 3577935...,INGeo: Accelerating Instant Neural Scene Recon...,2023,Lecture Notes in Computer Science (including s...,13803 LNCS,NaN,NaN,686,...,978-303125065-1,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,All Open Access; Green Open Access,Scopus,2-s2.0-85151135996
4,Magyar B.; Tokodi M.; Soós A.; Tolvaj M.; Laka...,"Magyar, Bálint (57733768900); Tokodi, Márton (...",57733768900; 57193751274; 57223141771; 5756118...,RVENet: A Large Echocardiographic Dataset for ...,2023,Lecture Notes in Computer Science (including s...,13803 LNCS,NaN,NaN,569,...,978-303125065-1,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85151119673


In [9]:
username ='priyankagupta@ms.du.ac.in'
password = 'Priya@1989'


In [10]:
print("logged in",file_.columns)

logged in Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'Molecular Sequence Numbers', 'Chemicals/CAS',
       'Tradenames', 'Manufacturers', 'Funding Details', 'Funding Texts',
       'References', 'Correspondence Address', 'Editors', 'Publisher',
       'Sponsors', 'Conference name', 'Conference date', 'Conference location',
       'Conference code', 'ISSN', 'ISBN', 'CODEN', 'PubMed ID',
       'Language of Original Document', 'Abbreviated Source Title',
       'Document Type', 'Publication Stage', 'Open Access', 'Source', 'EID'],
      dtype='object')


In [11]:
def get_name(html):
#     url = "https://www.scopus.com/authid/detail.uri?authorId=57205693172&amp;eid=2-s2.0-85061119789"
#     print url
#     driver.get(url) 
#     html = driver.page_source
    nameU = BeautifulSoup(html, "html.parser")
#     authInfo = nameU.find("div",{"class":"authInfoSection"})
    affil = nameU.find("div",{"class":"scopus-institution-name-link"})
    affiliation = affil.find("span")
#     country = affil.find(text=True,recursive=False)
#     print affil.text
    authN = nameU.find("h2",{"class":"author-general-details-title margin-size-4-t"})
    authName =''
    if authN:
        authName = authN.find(text=True,recursive =False)
    affiliat = ''
    if not authName:
        authName = ''
    if not affiliation:
        affiliat = ''
    else:
        affiliat = affiliation.text 
    print("Name",authName,affiliat)
    return(authName,affiliat)

In [12]:
abstract_ ='#profileleftinside > micro-ui > scopus-document-details-page > div > els-stack > article > div:nth-child(4) > section > div > div.margin-size-4-t.margin-size-16-b > els-typography > span'
author_keywords = '#profileleftinside > micro-ui > scopus-document-details-page > div > els-stack > article > div:nth-child(4) > section > div > div:nth-child(4)'
eng_controlled = '#indexed-keywords > section > div > div > els-stack > els-stack:nth-child(1) > dl > dd'
eng_uncontrolled = '#indexed-keywords > section > div > div > els-stack > els-stack:nth-child(2) > dl > dd'
eng_main_heading = '#indexed-keywords > section > div > div > els-stack > els-stack:nth-child(3) > dl > dd'
scival_tName ='#topics-of-prominence > section > div > div > div > dl > div.row.margin-size-12-b > dd > button > span'
scival_ = '#topics-of-prominence > section > div > div > div > dl > div:nth-child(2) > dd > span'


In [13]:
with open('PaperDetails_Gr2023_ECCV.csv',mode='a+') as dump:
    for index in range (0,file_.shape[0]):
    #     print(file_.iloc[index])
        link = file_.iloc[index]['Link']
        authors = str(file_.iloc[index]['Authors'])
        title = str(file_.iloc[index]['Title'])
        source_title = str(file_.iloc[index]['Source title'])
        year = str(file_.iloc[index]['Year'])
        aff = str(file_.iloc[index]['Author(s) ID'])
        AFFILATION = str(file_.iloc[index]['Affiliations'])
        PUBLISHER = str(file_.iloc[index]['Publisher'])
        cited_count = str(file_.iloc[index]['Cited by'])
    #     print("AFFILATION" + AFFILATION)
    #     print(type(year),type(aff),type(cited_count),type(link))

    #     print( index,link)
        driver.get(link)
    #     wait = WebDriverWait(driver, 10)
        time.sleep(6)
        try:
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            dict_ = {'Abstract':'','Author_Keywords':'','Eng_controlled':'','Eng_Uncontrolled':'',
                    'Eng_main_heading':'','Scival_topic':'','Scival_Score':-1, 'afflilations' :''}
            abstract = soup.select(abstract_)
            author_key =soup.select(author_keywords)
            eng_controlled_key = soup.select(eng_controlled)
            eng_uncontrolled_key = soup.select(eng_uncontrolled)
            eng_heading_key = soup.select(eng_main_heading)
            scival_tname = soup.select(scival_tName)
            scival_score = soup.select(scival_)
    #         aff_key = soup.select(afflilation)
            dict_['afflilations'] = AFFILATION
            dict_['publisher'] = PUBLISHER
            if len(abstract)>0:
                dict_['Abstract'] = abstract[0].text
            if len(author_key)>0:
                dict_['Author_Keywords'] = author_key[0].text
            if len(eng_controlled_key)>0:
                dict_['Eng_controlled'] = eng_controlled_key[0].text
            if len(eng_heading_key)>0:
                dict_['Eng_main_heading'] = eng_heading_key[0].text
            if len(eng_uncontrolled_key)>0:
                dict_['Eng_Uncontrolled'] = eng_uncontrolled_key[0].text
            if len(scival_tname)>0:
                dict_['Scival_topic'] = scival_tname[0].text
            if len(scival_score)>0:
                dict_['Scival_Score'] = scival_score[0].text
    #         print('found')
    #         Below code is Working . Uncomment if you want to extract author emails and affilations
    #         f = soup.select('#profileleftinside > micro-ui > scopus-document-details-page > div > els-stack > article > div:nth-child(1) > section > div:nth-child(3) > div > ul > li')
    #         print(len(f))
    #         for ind, li in enumerate(f):
    #             all_list =li.find_all('els-button')
    #             dict_={'AuthorName':'','Affiliation':'','Email':''}
    #             for m in all_list:
    #                 if m['variant']=='link':
    #                     dict_['AuthorName'] = m.text
    #                 if 'href' in list(m.attrs.keys()):
    #                     dict_['Email'] = m['href'].split('mailto:')[-1]

    #                     driver.find_element_by_css_selector('#profileleftinside > micro-ui > scopus-document-details-page > div > els-stack > article > div:nth-child(1) > section > div:nth-child(3) > div > ul > li:nth-child('+str(ind+1)+') > els-button').click()
    #                     page_source = driver.page_source
    #                     soup2 = BeautifulSoup(page_source,'html.parser')
    #                     affilation =soup2.select('#author-preview-flyout > div > div > div > div > div.flyout__dialog--content-slot > div > els-focus-trap > section > els-stack > els-stack.stack.stack--l.stack--mode-normal.stack--vertical.stack--start.hydrated > els-stack.stack.stack--s.stack--mode-normal.stack--vertical.stack--start.hydrated > els-stack > span > els-button')
    #     #                 print("affiliation",affilation,type(affilation))
    #                     driver.find_element_by_css_selector('#author-preview-flyout > div > div > div > div > div.flyout__dialog--content-slot > div > els-focus-trap > section > header > button > span.button__icon > els-icon > svg').click()
    #                     if len(affilation)==0:
    #                         dict_['Affiliation']=''
    #                     else:
    #                         dict_['Affiliation']=affilation[0].text
    #                 dfP  = pd.DataFrame([dict_], columns=dict_.keys())
    #                 with open('../Data/AuthorDetails_Gr2021_physics.csv',mode='a') as dump:
    #                     csv_data = re.sub(",","_",dict_['AuthorName'])+","+re.sub(","," ",dict_['Affiliation'])+","+re.sub(","," ",dict_['Email'])+","
    #                     print(csv_data)
    #                     dump_data = csv.writer(dump, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #                     dump_data.writerow(csv_data.split(","))

            print('FILE OPENED')
            csv_data = re.sub(",","_",authors)+","+re.sub(","," ",title)+","+re.sub(","," ",source_title)+","+re.sub(","," ",year)+","+re.sub(","," ",link)+","+re.sub(","," ",cited_count)
    #            +","+re.sub(",","_",dict_['Abstract'])+","+re.sub(","," ",dict_['Author_Keywords'])+","+re.sub(","," ",dict_['Eng_controlled'])+","+re.sub(","," ",dict_['Eng_Uncontrolled'])+","+re.sub(","," ",dict_['Eng_main_heading'])+","+re.sub(","," ",dict_['Scival_topic'])+ "," +
            csv_data += re.sub(","," ", dict_['publisher'])+ ","+re.sub(","," ", dict_['afflilations'])+ ","+re.sub(","," ",str(dict_['Scival_Score']))
    #            print(csv_data)
            try:
                print(dict_['afflilations'])
                new_entry = re.sub(","," ",ask_openai(base_prompt + dict_['afflilations']));
                print(new_entry)
                csv_data += "," + new_entry
            except:
                print("ERROR IN OPENAI")
                print(dict_['afflilations'])
            dump_data = csv.writer(dump, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            dump_data.writerow(csv_data.split(","))
        except:
            print(index,"error")

FILE OPENED
Computer Vision Lab, ETH Zürich, Zürich, Switzerland; AI Witchlabs, Zollikerberg, Switzerland; University of Wuerzburg, Wuerzburg, Germany; Ant Group, Hangzhou, China; ENERZAi, Seoul, South Korea; Zhejiang University of Technology, Hangzhou, China; Sensebrain Technology, San Jose, United States; Tetras.AI, Seoul, China; Korea Advanced Institute of Science and Technology (KAIST), Yuseong-gu, South Korea; Xiaomi Inc., Beijing, China
```json
{
  "ORGANIZATION": [
    "Computer Vision Lab" 
    "ETH Zürich" 
    "AI Witchlabs" 
    "University of Wuerzburg" 
    "Ant Group" 
    "ENERZAi" 
    "Zhejiang University of Technology" 
    "Sensebrain Technology" 
    "Tetras.AI" 
    "Korea Advanced Institute of Science and Technology (KAIST)" 
    "Xiaomi Inc."
  ] 
  "LOCATION": [
    "Zürich" 
    "Zollikerberg" 
    "Wuerzburg" 
    "Hangzhou" 
    "Seoul" 
    "San Jose" 
    "Yuseong-gu" 
    "Beijing"
  ] 
  "COUNTRY": [
    "Switzerland" 
    "Germany" 
    "China" 
    "Sou

FILE OPENED
CV Lab, University of Amsterdam, Amsterdam, Netherlands; AkzoNobel, Amsterdam, Netherlands; 3DUniversum, Amsterdam, Netherlands
{
    "ORGANIZATION": ["CV Lab"  "University of Amsterdam"  "AkzoNobel"  "3DUniversum"] 
    "LOCATION": ["Amsterdam"] 
    "COUNTRY": ["Netherlands"] 
    "MISCELLANEOUS": []
}
FILE OPENED
Computer Aided Medical Procedures, Technical University of Munich, Munich, Germany; Intuitive Surgical Inc., Sunnyvale, United States; Department of Psychiatry and Behavioral Sciences, Stanford University School of Medicine, Stanford, United States; Computer Aided Medical Procedures, Johns Hopkins University, Baltimore, United States
```json
{
    "ORGANIZATION": [
        "Computer Aided Medical Procedures" 
        "Technical University of Munich" 
        "Intuitive Surgical Inc." 
        "Department of Psychiatry and Behavioral Sciences" 
        "Stanford University School of Medicine" 
        "Johns Hopkins University"
    ] 
    "LOCATION": [
        "M

FILE OPENED
IDLab-AIRO, Ghent University – imec, Ghent, Belgium; RoboJob, Heist-op-den-Berg, Belgium; Department of Electromechanical, Systems and Metal Engineering, Ghent University, Ghent, Belgium; EEDT-DC, Flanders Make, Gent, Belgium; DecisionS, Flanders Make, Leuven, Belgium
{
    "ORGANIZATION": [
        "IDLab-AIRO" 
        "Ghent University" 
        "imec" 
        "RoboJob" 
        "Department of Electromechanical  Systems and Metal Engineering" 
        "Flanders Make" 
        "EEDT-DC" 
        "DecisionS"
    ] 
    "LOCATION": [
        "Ghent" 
        "Heist-op-den-Berg" 
        "Leuven" 
        "Gent"
    ] 
    "COUNTRY": [
        "Belgium"
    ] 
    "MISCELLANEOUS": []
}
FILE OPENED
Department of Computer Science Tandon School of Engineering, New York University New York, New York, 11202, United States; Center for Data Science New York University and Colton Center for Autoimmunity NYU Grossman School of Medicine New York, New York, 10011, United States
{
  "O

FILE OPENED
nan
{
  "ORGANIZATION": [] 
  "LOCATION": [] 
  "COUNTRY": [] 
  "MISCELLANEOUS": []
}
FILE OPENED
Department of Computer Science and Applied Mathematics, The Weizmann Institute of Science, Rehovot, Israel; Berkeley Artificial Intelligence Research (BAIR), Berkeley, United States
{
  "ORGANIZATION": [
    "Department of Computer Science and Applied Mathematics" 
    "The Weizmann Institute of Science" 
    "Berkeley Artificial Intelligence Research (BAIR)"
  ] 
  "LOCATION": [
    "Rehovot" 
    "Berkeley"
  ] 
  "COUNTRY": [
    "Israel" 
    "United States"
  ] 
  "MISCELLANEOUS": []
}
FILE OPENED
Department of Computer Science and Engineering, Southern University of Science and Technology, Shenzhen, China; University of Birmingham, Birmingham, United Kingdom; University of Electronic Science and Technology of China, Chengdu, China
{
  "ORGANIZATION": [
    "Department of Computer Science and Engineering" 
    "Southern University of Science and Technology" 
    "Universi

```json
{
  "ORGANIZATION": [
    "University of Ljubljana" 
    "University of Birmingham" 
    "Czech Technical University" 
    "Linköping University" 
    "Austrian Institute of Technology" 
    "TU Vienna" 
    "TU Munich" 
    "Technion Israel Institute of Technology" 
    "Tampere University" 
    "ETH Zurich" 
    "Umeå University" 
    "Academia Sinica" 
    "AI School" 
    "Beihang University" 
    "Beijing Jiaotong University" 
    "ByteDance" 
    "Dalian Maritime University" 
    "Dalian University of Technology" 
    "Dr B R Ambedkar National Institute of Technology Jalandhar" 
    "Guangxi Normal University" 
    "Huaqiao University" 
    "Indian Institute of Technology" 
    "Institute of Artificial Intelligence" 
    "Institute of Computing Technology" 
    "JD Finance America Corporation" 
    "Jiangnan University" 
    "LIVIA-École de technologie supérieure" 
    "Microsoft Research Asia" 
    "Nanjing University of Information Science and Technology" 
    "Nanjing 

...

FILE OPENED
School of Computer Science, Wuhan University, Wuhan, China; JD Explore Academy, Beijing, China; School of Computer Science, The University of Sydney, Sydney, Australia
{
  "ORGANIZATION": ["School of Computer Science"  "Wuhan University"  "JD Explore Academy"  "School of Computer Science"  "The University of Sydney"] 
  "LOCATION": ["Wuhan"  "Beijing"  "Sydney"] 
  "COUNTRY": ["China"  "China"  "Australia"] 
  "MISCELLANEOUS": []
}
FILE OPENED
SKLSDE, Beihang University, Beijing, China; SCSE, Beihang University, Beijing, China; CCST, NUAA, Nanjing, China; CNCERT/CC, Beijing, China
{
    "ORGANIZATION": ["SKLSDE"  "Beihang University"  "SCSE"  "CCST"  "NUAA"  "CNCERT/CC"] 
    "LOCATION": ["Beijing"  "Nanjing"  "Beijing"] 
    "COUNTRY": ["China"  "China"  "China"] 
    "MISCELLANEOUS": []
}
FILE OPENED
School of Engineering, The University of Tokyo, Tokyo, Japan
{
  "ORGANIZATION": [
    "School of Engineering" 
    "The University of Tokyo"
  ] 
  "LOCATION": [
    "Tokyo"

{
  "ORGANIZATION": ["Institute of Information Science"  "Beijing Jiaotong University"  "Beijing Key Laboratory of Advanced Information Science and Network Technology"  "Princeton University"  "JD Explore Academy"  "Sun Yat-sen University"] 
  "LOCATION": ["Beijing"  "Beijing"  "Beijing"  "Princeton"  "Beijing"  "Guangzhou"] 
  "COUNTRY": ["China"  "China"  "China"  "United States"  "China"  "China"] 
  "MISCELLANEOUS": []
}
FILE OPENED
School of Computing and Information Systems, The University of Melbourne, Parkville, VIC, Australia
{
  "ORGANIZATION": [
    "School of Computing and Information Systems" 
    "The University of Melbourne"
  ] 
  "LOCATION": [
    "Parkville"
  ] 
  "COUNTRY": [
    "Australia"
  ] 
  "MISCELLANEOUS": []
}
FILE OPENED
Chair of Robotics, Artificial Intelligence and Real-Time Systems, Technical University of Munich, Garching, 85748, Germany; Ohio State University, Columbus, 43212, OH, United States; Tongji University, Shanghai, 201804, China
{
  "ORGANIZ

FILE OPENED
University of Edinburgh, Edinburgh, United Kingdom; Meta AI, Menlo Park, United States
{
  "ORGANIZATION": ["University of Edinburgh"  "Meta AI"] 
  "LOCATION": ["Edinburgh"  "Menlo Park"] 
  "COUNTRY": ["United Kingdom"  "United States"] 
  "MISCELLANEOUS": []
}
FILE OPENED
College of Artificial Intelligence, Nankai University, Tianjin, 300350, China
{
    "ORGANIZATION": [
        "College of Artificial Intelligence" 
        "Nankai University"
    ] 
    "LOCATION": [
        "Tianjin"
    ] 
    "COUNTRY": [
        "China"
    ] 
    "MISCELLANEOUS": [
        "300350"
    ]
}
FILE OPENED
NEC Corporation, Kawasaki, Japan
{
  "ORGANIZATION": ["NEC Corporation"] 
  "LOCATION": ["Kawasaki"] 
  "COUNTRY": ["Japan"] 
  "MISCELLANEOUS": []
}
FILE OPENED
University of Virginia, Charlottesville, United States
{
  "ORGANIZATION": ["University of Virginia"] 
  "LOCATION": ["Charlottesville"] 
  "COUNTRY": ["United States"] 
  "MISCELLANEOUS": []
}
FILE OPENED
Show Lab, National

FILE OPENED
Illinois Institute of Technology, Chicago, United States; Hong Kong University of Science and Technology, Clear Water Bay, Hong Kong; Texas State University, San Marcos, United States; Harbin Institute of Technology, Shenzhen, China
{
  "ORGANIZATION": [
    "Illinois Institute of Technology" 
    "Hong Kong University of Science and Technology" 
    "Texas State University" 
    "Harbin Institute of Technology"
  ] 
  "LOCATION": [
    "Chicago" 
    "Clear Water Bay" 
    "San Marcos" 
    "Shenzhen"
  ] 
  "COUNTRY": [
    "United States" 
    "Hong Kong" 
    "China"
  ] 
  "MISCELLANEOUS": []
}
FILE OPENED
The College of Intelligence Science and Technology, National University of Defense Technology, Changsha, 410000, China
{
  "ORGANIZATION": [
    "College of Intelligence Science and Technology" 
    "National University of Defense Technology"
  ] 
  "LOCATION": [
    "Changsha"
  ] 
  "COUNTRY": [
    "China"
  ] 
  "MISCELLANEOUS": [
    "410000"
  ]
}
FILE OPENED
U

TimeoutException: Message: timeout: Timed out receiving message from renderer: -0.018
  (Session info: chrome-headless-shell=122.0.6261.69)
Stacktrace:
0   chromedriver                        0x00000001049fc53c chromedriver + 3966268
1   chromedriver                        0x00000001049f4ac8 chromedriver + 3934920
2   chromedriver                        0x0000000104677da0 chromedriver + 277920
3   chromedriver                        0x00000001046614f4 chromedriver + 185588
4   chromedriver                        0x0000000104661390 chromedriver + 185232
5   chromedriver                        0x000000010465fcf4 chromedriver + 179444
6   chromedriver                        0x000000010466082c chromedriver + 182316
7   chromedriver                        0x000000010466ef84 chromedriver + 241540
8   chromedriver                        0x00000001046820bc chromedriver + 319676
9   chromedriver                        0x0000000104660dbc chromedriver + 183740
10  chromedriver                    